In [2]:
import asyncio
import nest_asyncio
from ipyleaflet import Map, Marker, DivIcon, AwesomeIcon
from ipywidgets import VBox, HBox, Button, Layout
from IPython.display import display
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from model.model import SewerSystemModel  # Upewnij się, że ten import jest poprawny

# 1. Definicja funkcji, która PRZYJMUJE model jako argument
def run_sewer_visualization(model_instance, auto_interval=3):

    # 2. Model jest już stworzony, więc go tutaj nie tworzymy
    # model = SewerSystemModel(...)  <-- USUNIĘTE

    # Mapa (używamy przekazanego 'model_instance')
    m = Map(center=(49.684, 19.211), zoom=15)
    markers, labels = [], []

    # Tworzenie markerów
    def create_marker(agent, label, color):
        location = agent.location
        flow = getattr(agent, 'current_flow', getattr(agent, 'diverted_flow', 0))
        marker = Marker(
            location=location,
            draggable=False,
            icon=AwesomeIcon(name='circle', marker_color=color, icon_color='white')
        )
        label_marker = Marker(
            location=(location[0] + 0.0003, location[1]),
            icon=DivIcon(html=f"""
                <div style="font-size:13px;background:white;border:1px solid black;
                            padding:4px 6px;border-radius:6px;width:130px;text-align:center;
                            box-shadow:2px 2px 4px rgba(0,0,0,0.2);">
                    {label}<br>{flow} m³/h
                </div>
            """),
            draggable=False
        )
        return marker, label_marker

    # Dodanie punktów pomiarowych (używamy 'model_instance')
    for sensor in model_instance.sensors:
        if "Level4" in sensor.__class__.__name__:
            color = "green"
        elif "Level3" in sensor.__class__.__name__:
            color = "orange"
        else:
            color = "blue"

        m_sensor, l_sensor = create_marker(sensor, f"{sensor.location_id}", color)
        markers.append(m_sensor)
        labels.append(l_sensor)

    # Punkt przelewowy (używamy 'model_instance')
    over = model_instance.overflow_point
    m_over, l_over = create_marker(over, f"Overflow {over.location_id}", "red")
    markers.append(m_over)
    labels.append(l_over)

    # Oczyszczalnia (używamy 'model_instance')
    plant = model_instance.plant
    m_plant, l_plant = create_marker(plant, "Sewage Plant", "black")
    markers.append(m_plant)
    labels.append(l_plant)

    # Dodanie na mapę
    for mk, lb in zip(markers, labels):
        m.add(mk)
        m.add(lb)

    # Wykres
    fig = go.FigureWidget(make_subplots(rows=1, cols=1))
    fig.update_layout(
        title="Przepływy w czasie",
        xaxis_title="Godzina",
        yaxis_title="Przepływ (m³/h)",
        template="plotly_white",
        height=550
    )
    # Używamy 'model_instance' do pobrania limitu
    fig.add_hline(y=model_instance.max_capacity, line_dash="dash", line_color="red", annotation_text=f"Limit {model_instance.max_capacity} m³/h")

    step_btn = Button(description="⏸️ Pauza", button_style='info')
    m.layout = Layout(width='100%', height='550px')
    left_col = VBox([m, step_btn]); left_col.layout = Layout(width='65%', margin='10px')
    right_col = VBox([fig]);         right_col.layout = Layout(width='35%', margin='10px')
    dashboard = HBox([left_col, right_col]); dashboard.layout = Layout(width='100%', height='auto')

    # Wyświetl dashboard
    display(dashboard)
    print("Dashboard wyświetlony. Uruchamiam pętlę asynchroniczną...")

    running = True

    # Aktualizacja wizualizacji (funkcja wewnętrzna)
    async def update_visualization():
        nonlocal running
        try:
            # Używamy 'model_instance' w pętli
            while model_instance.running:
                if running:

                    # === DEBUG: Sprawdzamy, czy pętla działa ===
                    print(f"ASYNC STEP: Wykonuję krok dla godziny {model_instance.current_hour}")
                    # ==========================================

                    model_instance.step()

                    # Aktualizacja etykiet sensorów
                    for i, sensor in enumerate(model_instance.sensors):
                        flow = sensor.current_flow
                        color = "green" if sensor.status == "NORMAL" else "red"
                        markers[i].icon = AwesomeIcon(name='circle', marker_color=color, icon_color='white')
                        labels[i].icon = DivIcon(html=f"""
                            <div style="font-size:13px;background:white;border:1px solid black;
                                        padding:4px 6px;border-radius:6px;width:130px;text-align:center;
                                        box-shadow:2px 2px 4px rgba(0,0,0,0.2);">
                                {sensor.location_id}<br>{flow} m³/h
                            </div>
                        """)

                    # Aktualizacja przelewu
                    idx_over = len(model_instance.sensors)
                    over = model_instance.overflow_point
                    color_over = "red" if over.active else "gray"
                    markers[idx_over].icon = AwesomeIcon(name='tint', marker_color=color_over, icon_color='white')
                    labels[idx_over].icon = DivIcon(html=f"""
                        <div style="font-size:13px;background:white;border:1px solid black;
                                    padding:4px 6px;border-radius:6px;width:130px;text-align:center;
                                    box-shadow:2px 2px 4px rgba(0,0,0,0.2);">
                            Overflow {over.location_id}<br>{over.diverted_flow:.1f} m³/h
                        </div>
                    """)

                    # Aktualizacja oczyszczalni
                    labels[-1].icon = DivIcon(html=f"""
                        <div style="font-size:13px;background:white;border:1px solid black;
                                    padding:4px 6px;border-radius:6px;width:130px;text-align:center;
                                    box-shadow:2px 2px 4px rgba(0,0,0,0.2);">
                            Sewage Plant<br>Est. Flow: {model_instance.plant.estimated_flow:.1f} m³/h
                        </div>
                    """)

                    # Aktualizacja wykresu
                    df = model_instance.datacollector.get_model_vars_dataframe()
                    if not df.empty:
                        fig.data = [] # Czyści stary wykres
                        fig.add_trace(go.Scatter(
                            x=df.index, y=df["Estimated Flow"],
                            mode="lines+markers", name="Estimated Flow", line=dict(color="blue")
                        ))
                        if "Diverted Flow" in df.columns:
                            fig.add_trace(go.Scatter(
                                x=df.index, y=df["Diverted Flow"],
                                mode="lines+markers", name="Overflow Diverted", line=dict(color="red")
                            ))
                        fig.add_hline(y=model_instance.max_capacity, line_dash="dash", line_color="red")

                await asyncio.sleep(auto_interval)

            print(f"ASYNC: Pętla zakończona (model.running = {model_instance.running})")

        except Exception as e:
            print(f"!!!!!!!! PĘTLA ASYNC ZATRZYMANA PRZEZ BŁĄD !!!!!!!!!!")
            print(f"BŁĄD: {e}")
            import traceback
            traceback.print_exc()

    # Funkcja do pauzy
    def toggle_simulation(change):
        nonlocal running
        running = not running
        step_btn.description = "▶️ Wznów" if not running else "⏸️ Pauza"

    step_btn.on_click(toggle_simulation)

    # Uruchomienie pętli asynchronicznej
    nest_asyncio.apply()
    asyncio.ensure_future(update_visualization())
    print("Pętla asynchroniczna dodana do harmonogramu.")


# === KLUCZOWA SEKCJA URUCHOMIENIOWA ===

# 3. Stwórz model TUTAJ (globalnie w komórce), ustawiając max_hours na 7
print("Tworzę instancję modelu SewerSystemModel (max_hours=7)...")
model_globalny = SewerSystemModel(max_capacity=2000, max_hours=7)

# 4. URUCHOM funkcję wizualizacji, przekazując jej stworzony model
print("Uruchamiam funkcję run_sewer_visualization...")
run_sewer_visualization(model_globalny, auto_interval=3)

print("KONIEC KOMÓRKI. Wizualizacja powinna być aktywna.")

Tworzę instancję modelu SewerSystemModel (max_hours=7)...
Uruchamiam funkcję run_sewer_visualization...


Dashboard wyświetlony. Uruchamiam pętlę asynchroniczną...
Pętla asynchroniczna dodana do harmonogramu.
KONIEC KOMÓRKI. Wizualizacja powinna być aktywna.
ASYNC STEP: Wykonuję krok dla godziny 1

===== Godzina 1 =====
Limit nie zostanie przekroczony. Przewidywany poziom dopływu: 1100.0 m³/h
ASYNC STEP: Wykonuję krok dla godziny 2

===== Godzina 2 =====
Limit nie zostanie przekroczony. Przewidywany poziom dopływu: 1250.0 m³/h
ASYNC STEP: Wykonuję krok dla godziny 3

===== Godzina 3 =====
Limit nie zostanie przekroczony. Przewidywany poziom dopływu: 1517.0 m³/h
ASYNC STEP: Wykonuję krok dla godziny 4

===== Godzina 4 =====
Limit nie zostanie przekroczony. Przewidywany poziom dopływu: 1916.5 m³/h
ASYNC STEP: Wykonuję krok dla godziny 5

===== Godzina 5 =====
!!! Limit zostanie przekroczony! Przewidywany poziom dopływu: 2521.5 m³/h (limit: 2000)
Punkt przelewowy 10 otwarty → Do rzeki wpłynie 521.5 m³/h
ASYNC STEP: Wykonuję krok dla godziny 6

===== Godzina 6 =====
!!! Limit zostanie przekroc